In [1]:
import os
import sys
import datetime as dt

os.chdir("../Automated/")
from DataGathering import RedditScraper
from ChangePointAnalysis import ChangePointAnalysis
from NeuralNets import CreateNeuralNets


subreddit = "WallStreetBets"

Version 7.0.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lnajt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lnajt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lnajt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Scraping

For analyzing wallstreetbets data, we recommend downloading full.csv from [url] and putting it in ../Data/subreddit_wallstreetbets.

If you want to scrape a different subreddit, you can use the following file. You will need API.env with appropriate credentials in /Automated/

In [2]:

start = dt.datetime(2020, 1, 1)
end =  dt.datetime(2020, 1, 30)

if not os.path.exists(f"../Data/subreddit_{subreddit}/full.csv"):
    print("Did not find scraped data, scraping.")

    RedditScraper.scrape_data(subreddits = [subreddit], start = start, end = end)

# Change Point Analysis

The next cell will open full.csv , compute the words that are among the top daily_words most popular words on any day, and then run the change point analysis model on each of them.


The first time this is a run, a cleaned up version of the dataframe will be created for ease of processing.




In [ ]:
up_to = 2 # Only calculate change points for up_to of the popular words. Set to None to do all of them.
daily_words = 2 # Get the daily_words most popular posts on each day.

print('Computing the changepoints:')
ChangePointAnalysis.changepointanalysis([subreddit], up_to = up_to, daily_words = daily_words)


Computing the changepoints:
working on  WallStreetBets
Did not find preprocessed version, preprocessing. (This will only be done once.)
['debate', 'nio', 'virus', 'wsb', 'moon', 'biden', 'gme', 'sos', 'gang', 'merry', 'nkla', 'mt', 'two', 'week', 'tesla', 'tsla', 'go', 'currency', 'year', 'detroit', 'war', 'robinhood', 'calls', 'mascot', 'amc', 'azn', 'rh', 'citron', 'trump', 'mnmd', 'spy', 'wkhs', 'test', 'stock', 'pltr', 'spce', 'nikola', 'bezos', 'dd', 'like', 'mvis', 'ev', 'next', 'fortnite', 'kodk', 'bb', 'options', 'month', 'bear', 'election', 'new', 'slv', 'questrade', 'coin', 'stocks', 'christmas', 'short', 'covid', 'twitter', 'gold', 'dogecoin', 'hold', 'july', 'earnings', 'msft', 'today', 'get', 'one', 'fed', 'day', 'prpl', 'may', 'iran', 'sndl', 'trading', 'closed', 'rkt', 'tendies', 'tomorrow', 'trevor', 'buy', 'puts', 'im', 'suicide', 'snap', 'kodak', 'k', 'market', 'clov', 'plug', 'money', 'amd', 'monday', 'log', 'coronavirus', 'oil', 'apes', 'elon', 'uso', 'jpow'] pop_wo

C:\Users\lnajt\Documents\GitHub\ErdosInstitute\ErdosInstitute-SIG_Project\Automated\ChangePointAnalysis\BayesianMethods.py:134: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(steps, tune=tune, step = step)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [delta]
>Metropolis: [change_point_beta]
>Metropolis: [tau_beta]
>Metropolis: [beta_2]
>Metropolis: [alpha_2]
>Metropolis: [beta_1]
>Metropolis: [alpha_1]


After running, these files will in ../Data/subreddit_subreddit/Changepoints/Metropolis_30000Draws_5000Tune

(The final folder corresponds to the parameters of the Markov chain used by pymc3 for the inference.)

For instance: 
    
![title](../Data/subreddit_WallStreetBets/Changepoints/Metropolis_30000Draws_5000Tune/ChangePoint_tendies.png)

### Brief explanation of how this works:

The Bayesian model is as follows:

1. A coin is flipped with probability p.
2. If the coin comes up heads, then there is a change point. Otherwise, there is no change point.
3. It is assumed that the frequency random variable consists of independent draws from a beta distribution. If the coin decided there would be no change point, it is the same beta distribution at all times. Otherwise, it is a different beta on the different sides of the change points.

The posterior distribution of p is the models confidence that there is a change point, and the posterior distribution of tau represents its guess about when it occured.

Of course, this is not a realistic picture of the process; the independence of the different draws from the betas is especially unlike the data. However, it appears to be good enough to discover change points.

As currently written, it only handles one change point, however this can be improved.

In [7]:

DATA_FOLDER = f"../Data/subreddit_{subreddit}/Changepoints/metropolis_30000Draws_5000Tune/"
dataframe_path = f"../Data/subreddit_{subreddit}/"
preprocessed_location = dataframe_path + "changepoint_preprocessed.pkl"
pd.read_pickle(preprocessed_location)

UnpicklingError: invalid load key, ','.

# Neural Nets


In [ ]:
model, accuracies, word_tokenizer, df = CreateNeuralNets.buildnets([subreddit])[0]


## Predicted popularity as a time series

In [ ]:
text = "GME GME GME GME GME GME"
CreateNeuralNets.timeseries(df, text, model, word_tokenizer)